In [ ]:
import numpy as np
import copy
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
class Perceptron:
  def __init__(self, hidden_layers, activations, eta=1, n_epochs=100):
    self.layers = []
    self.epochs = n_epochs
    self.eta = eta
    self.errors = []
    l = hidden_layers + [1]
    for i in range(len(hidden_layers)):
      self.layers.append(l[i], activations[i], i)


  def predict(self, X, logging=False):
    result = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
      x = X[i, :]
      for layer in self.layers:
        x = np.append(x, values=[1]) # bias
        x = layer.forward(x, logging)
      result[i] = x[0]
      if logging:
        print(result[i])
    return result

  def train(self, X, y):
      ans = self.predict(X)
      for _ in range(self.epochs):
          error = 0
          for i in range(y):
              delta = y[i] - ans[i]
              error += delta
              for j in range(len(self.layers)):
                  self.layers[j].backward(self.eta*np.sign(y[i] != ans[i]))
          self.errors.append(error)
      return self.errors




class Layer:
  def __init__(self, size, activation, index):
    self.size = size
    self.w = np.zeros(self.size)
    self.activation = np.vectorize(activation)
    self.i = index

  def backward(self, x):
      self.w = self.w + x

  def forward(self, x, show_calc):
    m = np.dot(x, self.w)
    result = self.activation(m)

    if show_calc:
      print(f"Слой №{self.i+1}")
      print(f"Сенсоры  : {x}")
      print(f"Сумматор : {m}")
      print(f"Активация: {result}")

    return result